In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [ ]:
Empleados=pd.read_csv("https://raw.githubusercontent.com/AlcidesOxa/datos/master/ejemplo.csv",sep=';',encoding="ISO-8859-1")
Empleados.head()

In [ ]:
cambio={'Años estudio':'Anios_estudio', 'Estado Civil':'Estado_civil', 'Satisfacción Trabajo':'Satisfaccion_trabajo', 'Fecha contrato':'Fecha_contrato'}
Empleados.rename(columns=cambio,inplace=True)
casosborrar=[3,5,8]
Empleados=Empleados.drop(casosborrar)
varborrar=['Faltas','Permisos']
Empleados=Empleados.drop(varborrar,axis=1)
varcategoricas=['Sexo','Estado_civil','Satisfaccion_trabajo']
Empleados[varcategoricas]=Empleados[varcategoricas].fillna('Desconocido')
Empleados[varcategoricas]=Empleados[varcategoricas].astype('category')
Empleados['Fecha_contrato']=pd.to_datetime(Empleados.Fecha_contrato)
Empleados['Experiencia_lab']=Empleados.Edad-Empleados.Anios_estudio-5
Empleados['Balance']=Empleados.Ingresos-Empleados.Gastos

# Modelo de regresión lineal múltiple

El modelo clásico de regresión lineal múltiple es una técnica estadística, que permite explorar y confirmar relaciones lineales entre una variable de interés (dependiente) y un conjunto de variables explicativas (independientes), o predictoras de la variable dependiente. La relación no es completamente determinística, se admite un componente aleatorio o de perturbación, que captura la parte aleatoria de la variable dependiente, y que no es atribuible a las variables explicativas.

La especificación del modelo es
\begin{equation*}
Y_{i}=\beta _{1}+\beta _{2}X_{i2}+\beta _{3}X_{i3}+\ldots +\beta
_{k}X_{ik}+u_{i}
\end{equation*}
donde $Y_{i}$ ($i=1,2,\ldots ,n$) es la variable dependiente; $X_{i2}$, $X_{i3}$, $\ldots $, $X_{ik}$ son las variables explicativas; $\beta _{1}$, $\beta _{2}$, $\ldots $, $\beta _{k}$ son parámetros, con $\beta _{1}$ como la parte constante de $Y_{i}$ que no varía con las explictivas, también conocido como sesgo; $\beta _{2}$, $\ldots $, $\beta _{k}$ relacionan a la variable dependiente con las explicativas respectivas; y $u_{i}$ es un término de perturbación aleatoria.

El modelo en notación matricial es
\begin{eqnarray*}
Y &=&X\beta +u \\
\begin{pmatrix}
Y_{1} \\ 
Y_{2} \\ 
\vdots  \\ 
Y_{n}
\end{pmatrix}
&=&
\begin{pmatrix}
1 & X_{12} & \ldots  & X_{1k} \\ 
1 & X_{22} & \ldots  & X_{2k} \\ 
\vdots  & \vdots  & \ddots  & \vdots  \\ 
1 & X_{n2} & \ldots  & X_{nk}
\end{pmatrix}
\begin{pmatrix}
\beta _{1} \\ 
\beta _{2} \\ 
\vdots  \\ 
\beta _{k}
\end{pmatrix}
+
\begin{pmatrix}
u_{1} \\ 
u_{2} \\ 
\vdots  \\ 
u_{n}
\end{pmatrix}
\end{eqnarray*}
donde $n$ es el tamaño de la muestra, $k$ el número de variables y $n>k$. En la práctica, el vector de parámetros $β$ es desconocido. El objetivo es estimarlo a partir de una muestra aleatoria proveniente de la población objeto de estudio.

## Variables del modelo

In [ ]:
vars_mod = ['Ingresos', 'Edad', 'Sexo', 'Anios_estudio', 'Estado_civil']
vars_cat_mod = ['Sexo', 'Estado_civil']
vars_cuant_mod = ['Edad', 'Anios_estudio']
Datos = Empleados[vars_mod].dropna()
Datos.head()

## Análisis gráfico

In [ ]:
plt.scatter(Datos.Edad,Datos.Ingresos)

In [ ]:
plt.scatter(Datos.Anios_estudio,Datos.Ingresos)

In [ ]:
pd.plotting.scatter_matrix(Datos)

## Estimación por MCO

La contraparte muestral de $Y=X\beta +u$ con un estimador $\widehat{\beta }$ a determinar es
\begin{eqnarray*}
Y=X\widehat{\beta }+\widehat{u}
\end{eqnarray*}
y dado que tampoco se conoce $u$ se reemplaza por $\widehat{u}$. Aplicando la técnica de mínimos cuadrados ordinarios (MCO) se obtiene derivando parcialmente la suma de cuadrados $\widehat{u}^{\prime }\widehat{u}$ con respecto a los elementos del vector $\widehat{\beta }$ e igualando a $0$ (vector de ceros) para optimizar
\begin{eqnarray*}
\frac{\partial \widehat{u}^{\prime }\widehat{u}}{\partial \widehat{\beta }}=
\frac{\partial }{\partial \widehat{\beta }}\left( Y-X\widehat{\beta }\right)
^{\prime }\left( Y-X\widehat{\beta }\right) =-2X^{\prime }\left( Y-X\widehat{
\beta }\right) =0
\end{eqnarray*}
resolviendo ésta ecuación para $\widehat{\beta }$ se tiene el
estimador por MCO
\begin{eqnarray*}
\widehat{\beta }=\left( X^{\prime }X\right) ^{-1}X^{\prime }Y
\end{eqnarray*}

In [ ]:
X = Datos[vars_cuant_mod]
Y = Datos.Ingresos
reg = LinearRegression()
modelo1 = reg.fit(X,Y)
print(modelo1.coef_, ' ', modelo1.intercept_)

## Bondad de ajuste

El ajuste del modelo a los datos se mide por el coeficiente de determinación
\begin{equation*}
R^{2}=1-\frac{\sum_{i=1}^{n}\widehat{u}_{i}^{2}}{\sum_{i=1}^{n}\left( Y_{i}-\overline{Y}\right) ^{2}}
\end{equation*}
donde $\overline{Y}=\frac{1}{n}\sum_{i=1}^{n}Y_{i}$ es la media de la variable dependiente. Este coeficiente varía entre 0 y 1, si tiende a 0 entonces el modelo no se ajusta bien a los datos, por el contrario si se acerca a 1 entonces el ajuste es bueno. No obstante el $R^{2}$ no es un indicador adecuado sobre el ajuste del modelo fuera de la muestra de estimación, dado que es una función creciente de la cantidad de variables
explicativas, y puede aumentar aún con variables predictoras superfluas.
Como alternativa se propone el $R^{2}$ ajustado
\begin{equation*}
\overline{R}^{2}=1-\frac{n-1}{n-k}\left( 1-R^{2}\right) 
\end{equation*}
cuya interpretación es similar al del $R^{2}$, pero lo bueno es que las variables explicativas innecesarias no tienden a incrementarlo.

In [ ]:
R2 = modelo1.score(X,Y)
n, k = np.shape(X)
k = k + 1
R2a = 1 - (n-1)/(n-k)*(1-R2)
print(R2,' ',R2a)

## Predicción

La predicción de los valores de la media condicional de la variable dependiente es $\widehat{Y}=X\widehat{\beta }$. Para medir la incertidumbre de éstas predicciones se tiene la matriz de covarianzas de $\widehat{Y}$, y su estimación es
\begin{equation*}
\widehat{\Sigma }_{\widehat{y}}=\widehat{\sigma }^{2}X\left( X^{\prime
}X\right) ^{-1}X^{\prime }
\end{equation*}
Para evaluar las predicciones se aplica el error cuadrático medio (MSE por sus siglas en inglés) el cual es, en general, para cualquier estimación o predicción $\widehat{\theta }$
\begin{equation*}
MSE\left( \widehat{\theta }\right) =E\left[ \left( \widehat{\theta }-\theta \right) ^{2}\right]
\end{equation*}
Para las predicciones del modelo de regresión lineal, el MSE es
\begin{equation*}
MSE\left( \widehat{Y}\right)=\frac{\sum_{i=1}^{n}\left( Y_i-\widehat{Y_i} \right) ^{2}}{n}=\frac{\sum_{i=1}^{n}\widehat{u}_{i}^{2}}{n}
\end{equation*}
Mayores valores del MSE indican que los errores son grandes en la predicción, por el contrario, valores bajos indican una mejor predicción. En muchos casos se utiliza la raíz cuadrada del MSE para evaluar las predicciones, y entonces se habla del RMSE.

In [ ]:
Yp = modelo1.predict(X)
print('MSE', mean_squared_error(Y,Yp))

# Regresión polinómica y Pipelines

## Términos polinomiales

In [ ]:
W = [[3, 2], [5, 8]]
PolynomialFeatures(2).fit_transform(W)

## Pipelines

In [ ]:
pipe1 = Pipeline([
                  ('estand', StandardScaler()),
                  ('polin', PolynomialFeatures()),
                  ('reglin', LinearRegression())
])
pipe1.fit(X, Y)
pipe1.predict(X)

# Regresión con variables explicativas cualitativas

## Obtención de las variables dicotómicas

In [ ]:
Dicos = pd.get_dummies(Datos[vars_cat_mod], drop_first = True)
X = pd.concat([Dicos, Datos[vars_cuant_mod]], axis=1)
Y = Datos.Ingresos
X.head()

## Ajuste lineal y predicción

In [ ]:
reg = LinearRegression()
modelo1 = reg.fit(X,Y)
Yp = modelo1.predict(X)
print('MSE', mean_squared_error(Y,Yp), '-- Score', modelo1.score(X,Y))

# Regresión con variables dicotómicas y polinomiales

## Regresiones polinómicas

In [ ]:
for i in range(1,4):
  X = PolynomialFeatures(i).fit_transform(Datos[vars_cuant_mod])
  X = pd.DataFrame(X, index = Dicos.index.values)
  X = pd.concat([Dicos, X], axis=1)
  m1 = reg.fit(X, Y)
  Yp = m1.predict(X)
  print(mean_squared_error(Y, Yp))

## Regresiones polinómicas y evaluación fuera de la muestra

In [ ]:
DatosPrb = pd.DataFrame([[6240, 43, 'Hombre', 11, 'Casado(a)']], columns = vars_mod)
Datos_EP = pd.concat([Datos, DatosPrb], keys = ['Entr', 'Prb'])
Dicos_EP = pd.get_dummies(Datos_EP[vars_cat_mod], drop_first = True)
Y_e = Datos_EP.Ingresos.loc['Entr',:]
Y_p = Datos_EP.Ingresos.loc['Prb',:]
for i in range(1,4):
  X = PolynomialFeatures(i).fit_transform(Datos_EP[vars_cuant_mod])
  X = pd.DataFrame(X, index = Dicos_EP.index.values)
  X = pd.concat([Dicos_EP, X], axis=1)
  X_e = X.loc['Entr',:]
  X_p = X.loc['Prb',:]
  m1 = reg.fit(X_e, Y_e)
  Yp = m1.predict(X_p)
  print(mean_squared_error(Y_p, Yp))